# Imports


In [ ]:
! pip install evaluate

In [57]:
from datasets import Dataset, DatasetDict, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

# Load Data

In [58]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

# Load Pre-trained Model

In [59]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0:"Safe", 1:"Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}

In [60]:
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels = len(id2label),
                                                           id2label = id2label,
                                                           label2id = label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Set Trainable Parameters

In [61]:
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

for name, param in model.base_model.named_parameters():
  if 'pooler' in name:
    param.requires_grad = True

# Data Preprocessing

In [62]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation = True)

tokenized_data = dataset_dict.map(preprocess_function, batched = True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

In [63]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

# Evaluation Metrics

In [64]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

In [65]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred

  probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims = True)

  positive_class_probs = probabilities[:, 1]

  auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                   references=labels)['roc_auc'],3)

  predicted_classes = np.argmax(predictions, axis = 1)

  acc = np.round(accuracy.compute(predictions=predicted_classes,
                                     references=labels)['accuracy'],3)

  return {"Accuracy": acc, "AUC": auc}

# Training Parameters

In [71]:
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    logging_strategy = "epoch",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    report_to = "none"
)

# Fine-tune Model

In [72]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data["train"],
    eval_dataset = tokenized_data["test"],
    processing_class = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.488200,0.400082,0.811000,0.923000
2,0.398500,0.345953,0.842000,0.937000
3,0.372500,0.305958,0.867000,0.943000
4,0.351000,0.358601,0.842000,0.946000
5,0.344600,0.326083,0.856000,0.948000
6,0.326000,0.302504,0.876000,0.950000
7,0.303500,0.292095,0.871000,0.949000
8,0.324100,0.287452,0.876000,0.951000
9,0.313100,0.280607,0.878000,0.952000
10,0.302300,0.286853,0.871000,0.952000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=2630, training_loss=0.3523909188042122, metrics={'train_runtime': 6708.4817, 'train_samples_per_second': 3.13, 'train_steps_per_second': 0.392, 'total_flos': 706603239165360.0, 'train_loss': 0.3523909188042122, 'epoch': 10.0})

# Validation Data

In [73]:
predictions = trainer.predict(tokenized_data["validation"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'Accuracy': np.float64(0.873), 'AUC': np.float64(0.944)}
